In [ ]:
import math

from qiskit import QuantumCircuit
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution

from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session

In [ ]:
service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
def grover_oracle(marked_states):
    
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    for target in marked_states:
        rev_target = target[::-1]
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
    return qc

In [ ]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)
oracle.draw("mpl")

In [ ]:
grover_op = GroverOperator(oracle)
grover_op.decompose().draw("mpl")

In [ ]:
optimal_num_iterations = math.floor(
    math.pi / 4 * math.sqrt(2**grover_op.num_qubits / len(marked_states))
)

In [ ]:
qc = QuantumCircuit(grover_op.num_qubits)

qc.h(range(grover_op.num_qubits))
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
qc.measure_all()
qc.draw("mpl")

In [ ]:
# Amplitude Amplification

backend_simulator = service.least_busy(simulator=True, operational=True)
backend_simulator.name

In [ ]:
sim_session = Session(backend=backend_simulator)
sim_sampler = Sampler(session=sim_session)

In [ ]:
sim_dist = sim_sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]

In [ ]:
plot_distribution(sim_dist.binary_probabilities())

In [ ]:
sim_session.close()

In [ ]:
backend = service.least_busy(simulator=False, operational=True)
backend.name

In [ ]:
session = Session(backend=backend)
real_sampler = Sampler(session=session)

In [ ]:
real_dist = real_sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]

In [ ]:
plot_distribution(real_dist.binary_probabilities())

In [ ]:
session.close()